In [1]:
device ='cuda'

#Translating with T5

In [2]:
from transformers import pipeline
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-large")
model = model.to(device)

tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-large")



/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.13G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

In [3]:
import codecs
src_lines = []
file_name = 'medline_en-de.en.txt'

with codecs.open(file_name, 'r', 'utf-8') as src:
  for line in src:
    line = line.strip()
    src_lines.append(line)

In [ ]:
with open('translation_zeroshot_t5.txt', 'w', encoding='utf-8') as output_file:

  for src in src_lines:
    prompt = "translate English to German: {}.".format(src)


    inputs = tokenizer(prompt, return_tensors="pt").to(device)

    beam_output = model.generate(
        inputs["input_ids"],
        max_new_tokens=len(src),
        num_beams=5,
        early_stopping=True)

    translations = tokenizer.decode(beam_output[0], skip_special_tokens=True)

    output_file.write(translations + '\n')

#Translating with MBR

In [4]:
pip install mbr

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.3/106.3 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 17.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 21.1 MB/s eta 0:00:00


In [5]:
from transformers import T5Tokenizer, T5ForConditionalGeneration, AutoTokenizer
from mbr import MBR
from mbr import MBRGenerationMixin
from mbr import MBRConfig

In [6]:
class MBRt5Translation(MBRGenerationMixin, T5ForConditionalGeneration):
  pass

model = MBRt5Translation.from_pretrained("t5-large")
tokenizer = AutoTokenizer.from_pretrained("t5-large")

mbr_config = MBRConfig(num_samples =5, metric="chrf")


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.95G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/models/t5/tokenization_t5_fast.py:160: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-large automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


In [8]:

  for src in src_lines:
    prompt = "translate English to German: {}.".format(src)


    inputs = tokenizer(prompt, return_tensors="pt")

    output = model.generate(
        **inputs,
        max_new_tokens=len(src),
        tokenizer=tokenizer,
        mbr_config=mbr_config,
        do_sample = True,
        num_beams=1).to(device)

    translations = tokenizer.batch_decode(output, skip_special_tokens=True)
    print(translations)


['Alkoholische Hepatitis ist die akute Verschlechterung der alkoholischen Lebererkrankung bei schnellen Zuständen oder Verschlechterung der Sonnenbestrahlung, die in schweren Fällen zu einem Langzeit-acute-Leberversagen mit extrem hoher kurzzeitiger Sterblichkeit übergehen kann, wobei sich die Anzahl und die Schwere der hepaticen und extrahepatischen Organfunktionen beschleunigt.']
['Die Diagnose und Therapie sind unzureichend und sehr schwierig, insbesondere aufgrund der komplexen, multifaktoriellen und noch immer noch nicht vollständig verstandenen Pathogenese.']
['Während die Behandlung gegenwärtig auf steroid-ähnliche Medikamente und eine bestmögliche Unterstützungsversorgung beschränkt ist, ist eine Debatte über Lebertransplantationen ausgesuchter Patienten laufend, wobei mehrere neuartige Ansätze entwickelt werden, wobei das Ergebnis mit gemischten Ergebnissen war.']
['Diese Rücklagen im Krankheitsmanagement, vereint mit zunehmender Prävalenz in Deutschland, und allgemein in west

In [10]:

with open("translation_mbr.txt", "w", encoding="utf-8") as output_file:

    for src in src_lines:
        prompt = "translate English to German: {}.".format(src)

        inputs = tokenizer(prompt, return_tensors="pt")

        output = model.generate(
            **inputs,
            max_new_tokens=len(src),
            tokenizer=tokenizer,
            mbr_config=mbr_config,
            do_sample=True,
            num_beams=1
        ).to(device)

        translations = tokenizer.batch_decode(output, skip_special_tokens=True)

        # Write each translation to the file
        output_file.write(str(translations) + "\n")

In [2]:
#Removing unwanted characters before evaluation

with open("translation_mbr.txt", "r", encoding="utf-8") as input_file, open("translation_mbr_final.txt", "w", encoding="utf-8") as output_file:


    for line in input_file:
        translation = line[2:-3]
        output_file.write(translation + "\n")


#Evaluation

In [3]:
!pip install sacrebleu
!pip install unbabel-comet
import comet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.3/106.3 kB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.5/92.5 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 16.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.4/101.4 kB 13.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 32.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 777.7/777.7 kB 61.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 72.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 529.7/529.7 kB 51.1 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.3
    Uninstalling protobuf-3.20.3:
      Successfully uninstalled protobuf-3.20.3
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.20.1
    Uninstalling huggingface-hub-0.20.1:
      Successfully uninstalled huggingface-h

##Zeroshot T5 model

In [4]:
#compute BLEU and chrF scores
!sacrebleu medline_en-de.de.txt -l en-de -i translation_zeroshot_t5.txt  -m bleu chrf

[
{
 "name": "BLEU",
 "score": 14.5,
 "signature": "nrefs:1|case:mixed|eff:no|tok:13a|smooth:exp|version:2.4.0",
 "verbose_score": "38.9/17.4/10.3/6.3 (BP = 1.000 ratio = 1.043 hyp_len = 1281 ref_len = 1228)",
 "nrefs": "1",
 "case": "mixed",
 "eff": "no",
 "tok": "13a",
 "smooth": "exp",
 "version": "2.4.0"
},
{
 "name": "chrF2",
 "score": 39.3,
 "signature": "nrefs:1|case:mixed|eff:yes|nc:6|nw:0|space:no|version:2.4.0",
 "nrefs": "1",
 "case": "mixed",
 "eff": "yes",
 "nc": "6",
 "nw": "0",
 "space": "no",
 "version": "2.4.0"
}
]


In [7]:
!comet-score -s medline_en-de.en.txt -t translation_zeroshot_t5.txt -r medline_en-de_reference_de.txt

Seed set to 1
Fetching 5 files: 100% 5/5 [00:00<00:00, 15109.16it/s]
Lightning automatically upgraded your loaded checkpoint from v1.8.3.post1 to v2.1.3. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint ../root/.cache/huggingface/hub/models--Unbabel--wmt22-comet-da/snapshots/371e9839ca4e213dde891b066cf3080f75ec7e72/checkpoints/model.ckpt`
Encoder model frozen.
/usr/local/lib/python3.10/dist-packages/pytorch_lightning/core/saving.py:177: Found keys that are not in the model state dict but in the checkpoint: ['encoder.model.embeddings.position_ids']
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Predicting DataLoader 0: 100% 4/4 [00:02<00:00,  1.57it/s]
translation_zeroshot_t5.txt	Segment 0	score: 0.6059
translation_zeroshot_t5.txt	Segment 1	score: 0.8732
translation_zeroshot_t5.txt	

bleu: 14.5

chrf: 39.3

comet: 0.648

##MBR model

In [8]:
#compute BLEU and chrF scores
!sacrebleu medline_en-de.de.txt -l en-de -i translation_mbr_final.txt  -m bleu chrf

[
{
 "name": "BLEU",
 "score": 17.7,
 "signature": "nrefs:1|case:mixed|eff:no|tok:13a|smooth:exp|version:2.4.0",
 "verbose_score": "40.2/20.6/13.1/9.0 (BP = 1.000 ratio = 1.074 hyp_len = 1319 ref_len = 1228)",
 "nrefs": "1",
 "case": "mixed",
 "eff": "no",
 "tok": "13a",
 "smooth": "exp",
 "version": "2.4.0"
},
{
 "name": "chrF2",
 "score": 44.4,
 "signature": "nrefs:1|case:mixed|eff:yes|nc:6|nw:0|space:no|version:2.4.0",
 "nrefs": "1",
 "case": "mixed",
 "eff": "yes",
 "nc": "6",
 "nw": "0",
 "space": "no",
 "version": "2.4.0"
}
]


In [9]:
!comet-score -s medline_en-de.en.txt -t translation_mbr_final.txt -r medline_en-de_reference_de.txt

Seed set to 1
Fetching 5 files: 100% 5/5 [00:00<00:00, 58092.85it/s]
Lightning automatically upgraded your loaded checkpoint from v1.8.3.post1 to v2.1.3. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint ../root/.cache/huggingface/hub/models--Unbabel--wmt22-comet-da/snapshots/371e9839ca4e213dde891b066cf3080f75ec7e72/checkpoints/model.ckpt`
Encoder model frozen.
/usr/local/lib/python3.10/dist-packages/pytorch_lightning/core/saving.py:177: Found keys that are not in the model state dict but in the checkpoint: ['encoder.model.embeddings.position_ids']
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Predicting DataLoader 0: 100% 4/4 [00:03<00:00,  1.29it/s]
translation_mbr_final.txt	Segment 0	score: 0.6820
translation_mbr_final.txt	Segment 1	score: 0.8897
translation_mbr_final.txt	Segmen

bleu: 17.7

chrf: 44.4

comet: 0.669